# AMR SKID-STEERING PROJECT
### Notebook contents
* Kinodynamic RRT* implementation
### Notebook contributors
* Mattia Castelmare
* Andrea Giuseppe Di Francesco
* Enrico Fazzi

In [1]:
import numpy as np
import math
from env import *
from plotting import *
import copy
from utils import *
import matplotlib.pyplot as plt


In [92]:
from scipy import optimize
from RK4 import *
freq = 100
class KinoDynamicRRT_star:

    def __init__(self, qi, qf):

        self.qi = qi
        self.qf = qf

        self.A, self.B, self.R = self.get_AB(self.qf[3][0], self.qf[4][0], self.qf[2][0]) # Linearize around the final state qf


        self.q_d_0 = self.get_state(self.qf[3][0], self.qf[4][0], self.qf[2][0], 0, 0) # Computed with the zero inputs
        
        self.c = self.q_d_0 - self.A @ self.qf
      
        self.plan_opt_traj()
        
        
    def plan_opt_traj(self):

        print('COMPUTING OPTIMAL TIME TO TRAVERSE FROM Qi to Qf....\n')
        self.traj = self.get_tau_max()
        print('Tau optimal is: ', self.tau_m)

        # Compute the commands
        self.commands = self.get_u_func(self.traj)
        print(self.commands)
        self.states = self.reconstruct_states(self.traj, self.commands)
        print(self.states)


    def get_state(self, eta1, eta2, theta, u1, u2):
        ''' This function return the state of the system:
        
            args: eta1, eta2, theta, u1, u2: Configuration space;

            output: q = current state computed in eta1, eta2, theta, u1, u2
        '''

        q = np.array([[math.cos(theta)*eta1 - math.sin(theta)*eta2],
                    [math.sin(theta)*eta1 + math.cos(theta)*eta2],
                    [-(1/d0)*eta2],
                    [u1],
                    [u2]])
        
        return q


    def get_derivative(self, q, u):
        ''' Compute the Kinematic model in a configuration q with a control input u'''
        qd = self.A @ q + self.B @ u + self.c

        return qd
    
    def reconstruct_states(self, traj_time, u):
        ''' Reconstruct the input configurations with runge-kutta integration'''

        q = self.rungekutta4_state(self.get_derivative, self.qi, u, traj_time)

        return q
    

    def get_AB(self, eta1, eta2, theta):
        ''' This function return the A matrix that linearize the reduced kinematics in the skid-steering Robot :
        
            args: eta1, eta2, theta: Configuration space;

            output: A, B, R matrices
        '''

        A = np.array([[0, 0, -math.sin(theta)*eta1 - math.cos(theta)*eta2, math.cos(theta), -math.sin(theta)],
                    [0, 0, math.cos(theta)*eta1 - math.sin(theta)*eta2, math.sin(theta), math.cos(theta)],
                    [0, 0, 0, 0, -(1/d0)],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]])
        
        B = np.array([[0, 0],
                    [0, 0],
                    [0, 0],
                    [1, 0],
                    [0, 1]])
        
        R = np.array([[1, 0],
                    [0, 1]])
        
        return A, B, R



    def G_func(self, t):
        ''' This function computes "The weighted controllability Gramian", given the A, B and R matrix of a system
            for which we aim to minimize a trajectory w/ fixed-final-state-fixed-final-time. 
            
            args: A, B: Linearized dynamics of the skid-steering Robot ; R: Weights matrix of the control inputs ; t: independent variable 

            output: G_: not integrated Weighted controllability Gramian matrix

        '''

        G_d = lambda G_: self.A @ G_ + G_ @ self.A.T + self.B @ np.linalg.inv(self.R) @ self.B.T #transition function
        
        G_0 = np.zeros(self.A.shape)

        timesteps = np.arange(0, t, 1/freq)

        G = np.zeros((timesteps.shape[0], G_0.shape[0], G_0.shape[1]))


        for tim in range(timesteps.shape[0]):
            G_0 += np.exp(self.A*(t - timesteps[tim])) @ self.B @ np.linalg.inv(self.R) @ self.B.T @ np.exp(self.A.T*(t-timesteps[tim]))
            G[tim] = copy.copy(G_0)
        # G  = self.rungekutta4(G_d, G_0, t, g = True)

        # for dim in range(G.shape[0]):
        #     G[dim] = np.eye(G.shape[1])
        # G = G_0
        return G

    def x_bar(self, t):
        ''' This function computes the free evolution of the state x at time t with no control input in the current state 
        
            args: A, c, t: A: jacobian of the transition state function, c : Constant part of the transition state function, t: independent variable

            output: x_b
        
        '''

        x_b_d = lambda x_: self.A @ x_ + self.c

        x_b = self.rungekutta4(x_b_d, self.qi, t)
        # print(x_b[-1])
        return x_b

    def d_tau(self, t):
        ''' This function is a simplification for some formula that includes the Gramian matrix 
            G, x_b must be computed in the same time instant '''
        
        G = self.G_func(t)

        x_b = self.x_bar(t)
        

        d_t = np.linalg.pinv(G[int(t*freq) - 1]) @ (self.qf - x_b[int(t*freq) - 1])

        return d_t

    def rungekutta4(self, f, y0, tf, g = False):

        ''' This function integrates a transition function f, between 0 and tf, according to a sampling time 1/freq 
        
            args: f: transition function, y0: initial condition of the function that we want to integrate, tf: Final time instant

            output: y: Numpy array of the function integral, between 0 and tf
        
        '''

        if tf == 0:
            return y0
        
        t = np.arange(0, tf, 1/freq)

        y = np.zeros((t.shape[0], y0.shape[0], y0.shape[1]))

        y[0] = y0

        for i in range(t.shape[0] - 1):
            h = t[i+1] - t[i]
            k1 = f(y[i])
            k2 = f(y[i] + k1 * h / 2.)
            k3 = f(y[i] + k2 * h / 2.)
            k4 = f(y[i] + k3 * h)
            y[i+1] = y[i] + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)
            
            

        return y
    
    def rungekutta4_state(self, f, y0, u, traj_time):
        ''' This function integrates a transition function f, between 0 and tf, according to a sampling time 1/freq 
        
            args: f: transition function, y0: initial condition of the function that we want to integrate, tf: Final time instant

            output: y: Numpy array of the function integral, between 0 and tf
        
        '''
        

        y = np.zeros((traj_time.shape[0], y0.shape[0], y0.shape[1]))

        y[0] = y0

        for i in range(traj_time.shape[0] - 1):
            h = traj_time[i+1] - traj_time[i]
            k1 = f(y[i], u[i])
            k2 = f(y[i] + k1 * h / 2., u[i])
            k3 = f(y[i] + k2 * h / 2., u[i])
            k4 = f(y[i] + k3 * h, u[i])
            y[i+1] = y[i] + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)

        return y


    def compute_cost(self, t):
        ''' This function implements the cost function for the KINODYNAMIC RRT* 

            args: t: independant variable

            output: c_d: Cost function objective if the path last a time t
        
        '''
        G = self.G_func(t)
        x_b = self.x_bar(t)
        # print('x_b: ', x_b[int(t*freq) - 1])
        # print('qf: ', self.qf)
        # print('t: ', t) 
        # print('inv(G) ', np.linalg.inv(G[int(t*freq) - 1])) 
        # print('computation: ', np.linalg.inv(G[int(t*freq) - 1]) @ ((self.qf - x_b[int(t*freq) - 1])))
        # print('complete computation: ',(self.qf - x_b[int(t*freq) - 1]).T @ np.linalg.inv(G[int(t*freq) - 1]) @ ((self.qf - x_b[int(t*freq) - 1])))



        c = t + (self.qf - x_b[int(t*freq) - 1]).T @ np.linalg.pinv(G[int(t*freq) - 1]) @ ((self.qf - x_b[int(t*freq) - 1]))

        
        return c
    
    def cost_derivative(self, t):
        ''' This function implements the derivative of the cost function for the KINODYNAMIC RRT* 

            args: t: independant variable, x1: Final configuration, x0: Initial Configuration, A: A matrix of the system, B: B matrix of the system,
                R: Weight matrix of the control inputs, c: Constant values in the system/linearized system.

            output: c_d: Derivative of the cost function, we look for its roots in get_tau_max()
        
        '''

        d_t = self.d_tau(t)
        c_d = 1 - 2*(self.A @ self.qf + self.c).T @ d_t - d_t.T @ self.B @ np.linalg.inv(self.R) @ self.B.T @ d_t

        return c_d #[0][0]

    def get_tau_max(self):
        ''' This function uses an optimization library to compute the roots of the derivative's cost function, over a certain interval 

            args: LOOK AT THE COST DERIVATIVE FUNCTION

            output: tau_m: Minimum amount of time to have the minimum cost, time_traj: Time trajectories to compute the inputs and states.

        '''
        min_time_int = 0.00000001
        cost1 = self.cost_derivative(min_time_int)
        max_time_int = 0.2

        while cost1*self.cost_derivative(max_time_int) > 0:
            max_time_int += 0.1
            print(max_time_int)
        print('Found interval: ', max_time_int)

        sol = optimize.root_scalar(self.cost_derivative, bracket = [min_time_int, max_time_int], method = 'ridder')

        self.tau_m = sol.root

        time_traj = np.arange(0, self.tau_m, 1/freq)

        return time_traj

    def get_y_func(self, t, d_t):
        ''' This function computes the y function that is used to compute the control over time 
        
            args: t: independent variable,

            output: y value computed in t.
        
        '''

        y = np.exp(self.A.T*(self.tau_m - t)) @ d_t

        return y 

    def get_u_func(self, traj):
        ''' This function computes the u control function at the time t,
        
            args: t: independent variable,

            output: u value computed in t.
        
        '''
        d_t = self.d_tau(self.tau_m)
        commands = []

        for i in range(traj.shape[0]):
            y = self.get_y_func(traj[i], d_t)
            u = np.linalg.inv(self.R) @ self.B.T @ y
            commands.append(u)


        return commands

    

X_i = 1
Y_i = 1
theta_i = 1#math.pi/4 
eta1_i = 0
eta2_i = 0 


u1_i = 0.001
u2_i = 0.002


X_f = 1
Y_f = 1
theta_f = 1 #math.pi/4 
eta1_f = 0
eta2_f = 0


qi = np.array([[X_i], [Y_i], [theta_i], [eta1_i], [eta2_i]]) 

qf = np.array([[X_f], [Y_f], [theta_f], [eta1_f], [eta2_f]]) 




rrt = KinoDynamicRRT_star(qi, qf)




COMPUTING OPTIMAL TIME TO TRAVERSE FROM Qi to Qf....

0.30000000000000004
0.4
0.5
0.6
0.7
0.7999999999999999
0.8999999999999999
0.9999999999999999
1.0999999999999999
1.2
1.3
1.4000000000000001
1.5000000000000002
1.6000000000000003
1.7000000000000004
1.8000000000000005
1.9000000000000006
2.0000000000000004
2.1000000000000005
2.2000000000000006
2.3000000000000007
2.400000000000001
2.500000000000001
2.600000000000001
2.700000000000001
2.800000000000001
2.9000000000000012
3.0000000000000013
3.1000000000000014
3.2000000000000015
3.3000000000000016
3.4000000000000017
3.5000000000000018
3.600000000000002
3.700000000000002
3.800000000000002
3.900000000000002
4.000000000000002
4.100000000000001
4.200000000000001
4.300000000000001
4.4
4.5
4.6
4.699999999999999
4.799999999999999
4.899999999999999
4.999999999999998
5.099999999999998
5.1999999999999975
5.299999999999997
5.399999999999997
5.4999999999999964
5.599999999999996
5.699999999999996
5.799999999999995
5.899999999999995
5.999999999999995
6.0

KeyboardInterrupt: 

In [80]:
rrt.G_func(0.1)[-1]

[[19.74404439 20.63418838 17.45614756 19.85339248 19.85339248]
 [20.63418838 21.60009184 18.14508244 20.7796115  20.7796115 ]
 [17.45614756 18.14508244 15.7080695  17.4612884  17.4612884 ]
 [19.85339248 20.7796115  17.4612884  20.         20.        ]
 [19.85339248 20.7796115  17.4612884  20.         20.        ]]


array([[19.74404439, 20.63418838, 17.45614756, 19.85339248, 19.85339248],
       [20.63418838, 21.60009184, 18.14508244, 20.7796115 , 20.7796115 ],
       [17.45614756, 18.14508244, 15.7080695 , 17.4612884 , 17.4612884 ],
       [19.85339248, 20.7796115 , 17.4612884 , 20.        , 20.        ],
       [19.85339248, 20.7796115 , 17.4612884 , 20.        , 20.        ]])

In [ ]:
value = rrt.G_func(0.1)
np.linalg.pinv(value)

In [93]:
from scipy.optimize import minimize_scalar
minimize_scalar(rrt.compute_cost, method='bounded', bounds = [0, 100])

x_b:  [[ 5.58457932e+03]
 [ 1.27324865e+03]
 [-2.11111111e+02]
 [ 1.00000000e+00]
 [ 1.00000000e+00]]
qf:  [[1]
 [1]
 [1]
 [1]
 [1]]
t:  38.19660112501051


LinAlgError: Singular matrix